# Check against DIVD DB

Check if the NLM Hashes from the LEAK specific database are already in the DIVD database

# Set constants

In [ ]:
!ls ../LEAK_DB

In [ ]:
CASE="DIVD-2021-00012"
SUB="set1"

# Keep these the same
IN_DIR="../LEAK_DB/"
IN_DB=f"{IN_DIR}/{CASE}-{SUB}.sqlite3"
NML_DB="../SHARED_DBS/NML.sqlite3"

# Import helper functions

In [ ]:
%run ../0.shared_notebooks/0_helper_functions.ipynb

# Open DB

In [ ]:
nml_conn=sqlite3.connect(NML_DB)


In [ ]:
nml_conn


In [ ]:
in_conn=sqlite3.connect(IN_DB)

In [ ]:
in_conn

# Check hashes

In [ ]:
in_cur = in_conn.cursor()
in_cur.execute("SELECT COUNT( DISTINCT nml_hash) from entity;" )
row = in_cur.fetchone()
todo=row[0]
if todo > 0 :
    count = 0
    # Progress bar
    widgets=[
        progressbar.SimpleProgress(), 
        ' (', progressbar.Percentage(format='%(percentage)3.2f%%'), ') ',
        progressbar.Bar(),
        progressbar.ETA(),
    ]
    bar = progressbar.ProgressBar(max_value=todo, widgets=widgets, enable_colors=False)
    bar.update(1)
    for row in in_cur.execute("SELECT DISTINCT nml_hash FROM entity").fetchall() :
        if row[0] :
            for nmlrow in nml_conn.execute("""
                SELECT first, last 
                FROM hashes
                WHERE nml_hash = ?;
            """,
                ( row[0], )
            ).fetchall() :
                in_cur.execute(
                    "INSERT INTO nml ( nml_hash, first, last ) values ( ?, ?, ? );",
                    ( row[0], nmlrow[0], nmlrow[1] )
                )
        count=count+1
        if count % 100 == 0 :
            bar.update(count)
        if count % 10_000 == 0 :
            in_conn.commit()
    nml_conn.commit()
    in_conn.commit()
    print("Done")
    bar.update(todo)
else:
    print("Nothing to do here")

In [ ]:
todo

# Generate stats

In [ ]:
in_nml = 0
not_in_nnm=0
row = in_cur.execute('select count(*) from "entity" e left join nml n on e.nml_hash = n.nml_hash where e.nml_hash is not null and n.nml_hash is null collate nocase;').fetchone()
not_in_nml=row[0]
row = in_cur.execute('select count(*) from "entity" e left join nml n on e.nml_hash = n.nml_hash where e.nml_hash is not null and n.nml_hash is not null collate nocase;').fetchone()
in_nml=row[0]
rowcount=in_nml+not_in_nml
print(f"{rowcount:,} records, {in_nml:,} in nml ({in_nml/rowcount*100:.2f}%), {not_in_nml:,} not ({not_in_nml/rowcount*100:.2f}%)")

In [ ]:
row

In [ ]:
in_conn.commit()